In [ ]:
import numpy as np

def load_idx_file(filename):
    with open(filename, 'rb') as f:
        # 读取文件头
        magic_number = int.from_bytes(f.read(4), byteorder='big')
        num_items = int.from_bytes(f.read(4), byteorder='big')

        # 读取数据
        if magic_number == 2051:  # 图像数据
            num_rows = int.from_bytes(f.read(4), byteorder='big')
            num_cols = int.from_bytes(f.read(4), byteorder='big')
            data = np.frombuffer(f.read(), dtype=np.uint8)
            data = data.reshape(num_items, num_rows, num_cols)
        elif magic_number == 2049:  # 标签数据
            data = np.frombuffer(f.read(), dtype=np.uint8)
        else:
            raise ValueError("Unknown magic number for IDX file.")

    return data

# 指定本地MNIST数据集路径
mnist_path = '/kaggle/input/mnist-data/MNIST_data'

# 加载训练集和测试集
x_train = load_idx_file(f'{mnist_path}/train-images-idx3-ubyte/train-images.idx3-ubyte')  
y_train = load_idx_file(f'{mnist_path}/train-labels-idx1-ubyte/train-labels.idx1-ubyte')   
x_test = load_idx_file(f'{mnist_path}/t10k-images-idx3-ubyte/t10k-images.idx3-ubyte')  
y_test = load_idx_file(f'{mnist_path}/t10k-labels-idx1-ubyte/t10k-labels.idx1-ubyte') 




In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 将图像数据转换为张量，并进行标准化
x_train_tensor = torch.Tensor(x_train).unsqueeze(1) / 255.0
y_train_tensor = torch.Tensor(y_train).long()
x_test_tensor = torch.Tensor(x_test).unsqueeze(1) / 255.0
y_test_tensor = torch.Tensor(y_test).long()

# 创建数据集和数据加载器
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#搭建深度自编码器网络
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 2),  # bottleneck layer with 2 neurons
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 初始化模型、损失函数和优化器
autoencoder = Autoencoder()
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)


# 定义噪声因子
noise_factor = 0.4

# 训练模型
num_epochs = 100
for epoch in range(num_epochs):
    for data, _ in train_loader:
        # 在输入图像上添加高斯噪声
        noisy_data = data + noise_factor * torch.randn(data.shape)

        optimizer.zero_grad()
        outputs = autoencoder(noisy_data)
        loss = criterion(outputs, data.view(-1, 28 * 28))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 评估模型
autoencoder.eval()

# 测试集的一个批次
with torch.no_grad():
    test_data, _ = next(iter(test_loader))
    # 在测试图像上添加高斯噪声
    noisy_test_data = test_data + noise_factor * torch.randn(test_data.shape)
    test_outputs = autoencoder(noisy_test_data)


# 显示原始图像、噪声图像和降噪重建图像
n = 10
fig, axes = plt.subplots(3, n, figsize=(20, 6))

for i in range(n):
    # 原始图像
    axes[0, i].imshow(test_data[i].numpy().reshape(28, 28), cmap='gray')
    axes[0, i].get_xaxis().set_visible(False)
    axes[0, i].get_yaxis().set_visible(False)
    axes[0, i].set_title('')

    # 噪声图像
    axes[1, i].imshow(noisy_test_data[i].numpy().reshape(28, 28), cmap='gray')
    axes[1, i].get_xaxis().set_visible(False)
    axes[1, i].get_yaxis().set_visible(False)
    axes[1, i].set_title('')

    # 降噪重建图像
    axes[2, i].imshow(test_outputs[i].numpy().reshape(28, 28), cmap='gray')
    axes[2, i].get_xaxis().set_visible(False)
    axes[2, i].get_yaxis().set_visible(False)
    axes[2, i].set_title('')

plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 将图像数据转换为张量，并进行标准化
x_train_tensor = torch.Tensor(x_train).unsqueeze(1) / 255.0
y_train_tensor = torch.Tensor(y_train).long()
x_test_tensor = torch.Tensor(x_test).unsqueeze(1) / 255.0
y_test_tensor = torch.Tensor(y_test).long()

# 创建数据集和数据加载器
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

batch_size = 64
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

#搭建深度自编码器网络
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 2),  # bottleneck layer with 2 neurons
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid(),
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 初始化模型、损失函数和优化器
autoencoder = Autoencoder()
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for data, _ in train_loader:
        optimizer.zero_grad()
        outputs = autoencoder(data)
        loss = criterion(outputs, data.view(-1, 28 * 28))
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')




In [ ]:
import matplotlib.pyplot as plt
import torch

# 评估模型
autoencoder.eval()

# 获取测试集中的图像和标签
with torch.no_grad():
    test_data, test_labels = next(iter(test_loader))
    encoded_data = autoencoder.encoder(test_data.view(test_data.size(0), -1))

# 对 Latent code 进行均匀采样
num_samples = 100
latent_samples = torch.rand((num_samples, 2))  # 2是 bottleneck layer 的维度

# 利用解码器对采样结果进行恢复
with torch.no_grad():
    decoded_samples = autoencoder.decoder(latent_samples)

# 显示 Latent space 中的数字聚类情况
plt.figure(figsize=(8, 6))
plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=test_labels.numpy(), cmap='viridis')
plt.title('Latent Space Visualization')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.colorbar()
plt.show()

# 使用解码器将 Latent space 中的采样数字重构为图像
plt.figure(figsize=(12, 4))
for i in range(num_samples):
    plt.subplot(10, 10, i + 1)
    plt.imshow(decoded_samples[i].view(28, 28).numpy(), cmap='gray')
    plt.title(f'')
    plt.axis('off')
plt.show()


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义深度自编码器模型
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 8),
            nn.ReLU(),
            nn.Linear(8, 2),  # bottleneck layer with 2 neurons
        )
        self.decoder = nn.Sequential(
            nn.Linear(2, 8),
            nn.ReLU(),
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 256),
            nn.ReLU(),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid()  # 输出范围在 [0, 1]
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

# 初始化模型、损失函数和优化器（添加权重衰减项）
autoencoder = Autoencoder()
criterion = nn.BCELoss()
optimizer = optim.Adam(autoencoder.parameters(), lr=0.001, weight_decay=1e-5)  # 添加权重衰减项

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    for data, _ in train_loader:
        optimizer.zero_grad()
        outputs = autoencoder(data)
        loss = criterion(outputs, data.view(-1, 28 * 28))
        
        # 添加 L2 正则化项
        l2_regularization = sum(torch.sum(param**2) for param in autoencoder.parameters())
        loss += 1e-5 * l2_regularization
        
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

In [ ]:
import matplotlib.pyplot as plt
import torch

# 评估模型
autoencoder.eval()

# 获取测试集中的图像和标签
with torch.no_grad():
    test_data, test_labels = next(iter(test_loader))
    encoded_data = autoencoder.encoder(test_data.view(test_data.size(0), -1))

# 对 Latent code 进行均匀采样
num_samples = 100
latent_samples = torch.rand((num_samples, 2))  # 2是 bottleneck layer 的维度

# 利用解码器对采样结果进行恢复
with torch.no_grad():
    decoded_samples = autoencoder.decoder(latent_samples)

# 显示 Latent space 中的数字聚类情况
plt.figure(figsize=(8, 6))
plt.scatter(encoded_data[:, 0], encoded_data[:, 1], c=test_labels.numpy(), cmap='viridis')
plt.title('Latent Space Visualization')
plt.xlabel('Latent Dimension 1')
plt.ylabel('Latent Dimension 2')
plt.colorbar()
plt.show()

# 使用解码器将 Latent space 中的采样数字重构为图像
plt.figure(figsize=(12, 4))
for i in range(num_samples):
    plt.subplot(10, 10, i + 1)
    plt.imshow(decoded_samples[i].view(28, 28).numpy(), cmap='gray')
    plt.title(f'')
    plt.axis('off')
plt.show()
